In [1]:
from pyspark.sql import  SQLContext, Row
from pyspark import SparkContext,SparkConf
import pandas as pd
from pyspark.sql.functions import regexp_extract, regexp_replace, when,udf,col,count,sum,avg,round
from pyspark.sql.types import DoubleType,IntegerType,StringType

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc )

In [3]:
df1 = sqlCtx.read.csv('data/2014년 졸음운전 교통사고.csv', inferSchema=True, header=True,encoding='euc-kr')
df2 = sqlCtx.read.csv('data/2015년 졸음운전 교통사고.csv', inferSchema=True, header=True,encoding='euc-kr')
df3 = sqlCtx.read.csv('data/2016년 졸음운전 교통사고.csv', inferSchema=True, header=True,encoding='euc-kr')

In [5]:
df1.show()

+----------+--------+--------+--------+
|      구분|사고(건)|사망(명)|부상(명)|
+----------+--------+--------+--------+
| 2014년1월|     169|       7|     347|
| 2014년2월|     145|       3|     268|
| 2014년3월|     206|      12|     358|
| 2014년4월|     182|       4|     360|
| 2014년5월|     218|      17|     474|
| 2014년6월|     233|      13|     457|
| 2014년7월|     229|      14|     431|
| 2014년8월|     222|       9|     447|
| 2014년9월|     225|      15|     430|
|2014년10월|     210|      14|     392|
|2014년11월|     209|      10|     386|
|2014년12월|     178|      12|     329|
+----------+--------+--------+--------+



In [4]:
df = df1.unionAll(df2)
df = df.unionAll(df3)
df.show(df.count())

+----------+--------+--------+--------+
|      구분|사고(건)|사망(명)|부상(명)|
+----------+--------+--------+--------+
| 2014년1월|     169|       7|     347|
| 2014년2월|     145|       3|     268|
| 2014년3월|     206|      12|     358|
| 2014년4월|     182|       4|     360|
| 2014년5월|     218|      17|     474|
| 2014년6월|     233|      13|     457|
| 2014년7월|     229|      14|     431|
| 2014년8월|     222|       9|     447|
| 2014년9월|     225|      15|     430|
|2014년10월|     210|      14|     392|
|2014년11월|     209|      10|     386|
|2014년12월|     178|      12|     329|
| 2015년1월|     205|       8|     375|
| 2015년2월|     175|       8|     375|
| 2015년3월|     237|       4|     492|
| 2015년4월|     227|      10|     486|
| 2015년5월|     231|       9|     504|
| 2015년6월|     207|       8|     401|
| 2015년7월|     273|       7|     556|
| 2015년8월|     268|      19|     534|
| 2015년9월|     252|      10|     501|
|2015년10월|     214|      14|     475|
|2015년11월|     199|       3|     398|
|2015년12월|     21

In [6]:
df.printSchema()

root
 |-- 구분: string (nullable = true)
 |-- 사고(건): integer (nullable = true)
 |-- 사망(명): integer (nullable = true)
 |-- 부상(명): integer (nullable = true)



In [ ]:
# https://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html

In [7]:
from pyspark.sql.functions import to_timestamp,date_format,dayofweek

In [9]:
#2016/12/13  : yyyy/MM/dd
#2016_12_13 12:12:13  : yyyy_MM_dd HH:mm:ss
df = df.withColumn( 'tmparse', to_timestamp('구분', "yyyy년M월") )
df.show()

+----------+--------+--------+--------+-------------------+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|
+----------+--------+--------+--------+-------------------+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00|
| 2015년2월|     175|       8|     375|2015-02-01 00:00:00|
| 2015년3월|     237|

In [10]:
df.printSchema()

root
 |-- 구분: string (nullable = true)
 |-- 사고(건): integer (nullable = true)
 |-- 사망(명): integer (nullable = true)
 |-- 부상(명): integer (nullable = true)
 |-- tmparse: timestamp (nullable = true)



In [11]:
df.where( df['tmparse']>='2015' ).show()

+----------+--------+--------+--------+-------------------+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|
+----------+--------+--------+--------+-------------------+
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00|
| 2015년2월|     175|       8|     375|2015-02-01 00:00:00|
| 2015년3월|     237|       4|     492|2015-03-01 00:00:00|
| 2015년4월|     227|      10|     486|2015-04-01 00:00:00|
| 2015년5월|     231|       9|     504|2015-05-01 00:00:00|
| 2015년6월|     207|       8|     401|2015-06-01 00:00:00|
| 2015년7월|     273|       7|     556|2015-07-01 00:00:00|
| 2015년8월|     268|      19|     534|2015-08-01 00:00:00|
| 2015년9월|     252|      10|     501|2015-09-01 00:00:00|
|2015년10월|     214|      14|     475|2015-10-01 00:00:00|
|2015년11월|     199|       3|     398|2015-11-01 00:00:00|
|2015년12월|     213|       8|     428|2015-12-01 00:00:00|
| 2016년1월|     192|       5|     387|2016-01-01 00:00:00|
| 2016년2월|     174|       6|     328|2016-02-01 00:00:00|
| 2016년3월|     217|

In [12]:
df.where( df['tmparse']>='2015-05' ).show()

+----------+--------+--------+--------+-------------------+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|
+----------+--------+--------+--------+-------------------+
| 2015년5월|     231|       9|     504|2015-05-01 00:00:00|
| 2015년6월|     207|       8|     401|2015-06-01 00:00:00|
| 2015년7월|     273|       7|     556|2015-07-01 00:00:00|
| 2015년8월|     268|      19|     534|2015-08-01 00:00:00|
| 2015년9월|     252|      10|     501|2015-09-01 00:00:00|
|2015년10월|     214|      14|     475|2015-10-01 00:00:00|
|2015년11월|     199|       3|     398|2015-11-01 00:00:00|
|2015년12월|     213|       8|     428|2015-12-01 00:00:00|
| 2016년1월|     192|       5|     387|2016-01-01 00:00:00|
| 2016년2월|     174|       6|     328|2016-02-01 00:00:00|
| 2016년3월|     217|       7|     435|2016-03-01 00:00:00|
| 2016년4월|     216|       7|     419|2016-04-01 00:00:00|
| 2016년5월|     239|      13|     522|2016-05-01 00:00:00|
| 2016년6월|     200|      12|     362|2016-06-01 00:00:00|
| 2016년7월|     227|

In [13]:
df.withColumn( 'year', date_format(df['tmparse'], 'yyyy' ) ).show()

+----------+--------+--------+--------+-------------------+----+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|year|
+----------+--------+--------+--------+-------------------+----+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|2014|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|2014|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|2014|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|2014|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|2014|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|2014|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|2014|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|2014|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|2014|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|2014|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|2014|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|2014|
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00|201

In [14]:
df.withColumn( 'month', date_format(df['tmparse'], 'MM' ) ).show()

+----------+--------+--------+--------+-------------------+-----+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|month|
+----------+--------+--------+--------+-------------------+-----+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|   01|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|   02|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|   03|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|   04|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|   05|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|   06|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|   07|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|   08|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|   09|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|   10|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|   11|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|   12|
| 2015년1월|     205|       8|     375|2015-01-

In [17]:
df.withColumn( 'day', date_format(df['tmparse'], 'dd' ) ).show()

+----------+--------+--------+--------+-------------------+---+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|day|
+----------+--------+--------+--------+-------------------+---+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00| 01|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00| 01|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00| 01|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00| 01|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00| 01|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00| 01|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00| 01|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00| 01|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00| 01|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00| 01|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00| 01|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00| 01|
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00| 01|
| 2015년2월|   

In [19]:
df.withColumn( 'year&month', date_format(df['tmparse'], 'yyyy MM' ) ).show()

+----------+--------+--------+--------+-------------------+----------+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|year&month|
+----------+--------+--------+--------+-------------------+----------+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|   2014 01|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|   2014 02|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|   2014 03|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|   2014 04|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|   2014 05|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|   2014 06|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|   2014 07|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|   2014 08|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|   2014 09|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|   2014 10|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|   2014 11|
|2014년12월|     178|      12|     329|20

In [20]:
df.withColumn( 'f1', date_format( df['tmparse'], 'yyyy MM dd EEEE' ) ).show()

+----------+--------+--------+--------+-------------------+--------------------+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|                  f1|
+----------+--------+--------+--------+-------------------+--------------------+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|2014 01 01 Wednesday|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00| 2014 02 01 Saturday|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00| 2014 03 01 Saturday|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  2014 04 01 Tuesday|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00| 2014 05 01 Thursday|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|   2014 06 01 Sunday|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  2014 07 01 Tuesday|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|   2014 08 01 Friday|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|   2014 09 01 Monday|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|

In [21]:
# df.withColumn( 'year', date_format(df['tmparse'], 'yyyy' ) ).show()
# df.withColumn( 'month', date_format(df['tmparse'], 'MM' ) ).show()
# df.withColumn( 'day', date_format(df['tmparse'], 'dd' ) ).show()
# df.withColumn( 'hour', date_format(df['tmparse'], 'HH' ) ).show()
# df.withColumn( 'year&month', date_format(df['tmparse'], 'yyyy MM' ) ).show()
# df.withColumn( 'month', date_format(df['tmparse'], 'yyyy MM dd EEEE' ) ).show()
# df.withColumn( 'month', date_format(df['tmparse'], 'W' ) ).show() -> 없어짐
# dayofweek 1:일 2:월 3:화 4:수 5:목 6:금 7:토
df.withColumn( 'f1', date_format( df['tmparse'], 'yyyy MM dd EEEE' ) ).\
    withColumn( 'f2', dayofweek( col('tmparse') ) ).show()
# df['tmparse'] = col('tmparse')

+----------+--------+--------+--------+-------------------+--------------------+---+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|                  f1| f2|
+----------+--------+--------+--------+-------------------+--------------------+---+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|2014 01 01 Wednesday|  4|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00| 2014 02 01 Saturday|  7|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00| 2014 03 01 Saturday|  7|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  2014 04 01 Tuesday|  3|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00| 2014 05 01 Thursday|  5|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|   2014 06 01 Sunday|  1|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  2014 07 01 Tuesday|  3|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|   2014 08 01 Friday|  6|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|   2014 09 01 Monday|  2|
|2014년10월

In [22]:
df = df.withColumn( 'w', dayofweek( col('tmparse') ) )
df.show()

+----------+--------+--------+--------+-------------------+---+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|  w|
+----------+--------+--------+--------+-------------------+---+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|  4|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|  7|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|  7|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  3|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|  5|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|  1|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  3|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|  6|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|  2|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|  4|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|  7|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|  2|
| 2015년1월|     205|       8|     375|2015-01-01 00:00:00|  5|
| 2015년2월|   

In [24]:
from pyspark.sql.functions import udf
w ={1:'일',2:'월',3:'화',4:'수',5:'목',6:'금',7:'토'}
ff = udf(  lambda v: w[v] )
df.withColumn( '요일', ff( df['w'] ) ).show()

+----------+--------+--------+--------+-------------------+---+----+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|  w|요일|
+----------+--------+--------+--------+-------------------+---+----+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|  4|  수|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|  7|  토|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|  7|  토|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  3|  화|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|  5|  목|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|  1|  일|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  3|  화|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|  6|  금|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|  2|  월|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|  4|  수|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|  7|  토|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|  2|  월|
| 2015년1월|    

+----------+--------+--------+--------+-------------------+---+----+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|  w|요일|
+----------+--------+--------+--------+-------------------+---+----+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|  4|  수|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|  7|  토|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|  7|  토|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  3|  화|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|  5|  목|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|  1|  일|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  3|  화|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|  6|  금|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|  2|  월|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|  4|  수|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|  7|  토|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|  2|  월|
| 2015년1월|    

In [25]:
df.withColumn('year',date_format(df['tmparse'], 'yyyy')).show()

+----------+--------+--------+--------+-------------------+---+----+
|      구분|사고(건)|사망(명)|부상(명)|            tmparse|  w|year|
+----------+--------+--------+--------+-------------------+---+----+
| 2014년1월|     169|       7|     347|2014-01-01 00:00:00|  4|2014|
| 2014년2월|     145|       3|     268|2014-02-01 00:00:00|  7|2014|
| 2014년3월|     206|      12|     358|2014-03-01 00:00:00|  7|2014|
| 2014년4월|     182|       4|     360|2014-04-01 00:00:00|  3|2014|
| 2014년5월|     218|      17|     474|2014-05-01 00:00:00|  5|2014|
| 2014년6월|     233|      13|     457|2014-06-01 00:00:00|  1|2014|
| 2014년7월|     229|      14|     431|2014-07-01 00:00:00|  3|2014|
| 2014년8월|     222|       9|     447|2014-08-01 00:00:00|  6|2014|
| 2014년9월|     225|      15|     430|2014-09-01 00:00:00|  2|2014|
|2014년10월|     210|      14|     392|2014-10-01 00:00:00|  4|2014|
|2014년11월|     209|      10|     386|2014-11-01 00:00:00|  7|2014|
|2014년12월|     178|      12|     329|2014-12-01 00:00:00|  2|2014|


In [26]:
#년도별 사망(명)의 평균을 구하시요
df.withColumn('year',date_format(df['tmparse'], 'yyyy')).groupBy('year').avg('사망(명)').show()

+----+------------------+
|year|     avg(사망(명))|
+----+------------------+
|2016| 8.166666666666666|
|2014|10.833333333333334|
|2015|               9.0|
+----+------------------+



In [32]:
#년도 및 월별 사망 평균
df.withColumn('year',date_format(df['tmparse'], 'yyyy')).\
withColumn('month',date_format(df['tmparse'], 'MM')).\
groupBy('year', 'month').avg('사망(명)').show()

+----+-----+-------------+
|year|month|avg(사망(명))|
+----+-----+-------------+
|2015|   09|         10.0|
|2016|   05|         13.0|
|2014|   11|         10.0|
|2014|   12|         12.0|
|2015|   11|          3.0|
|2014|   03|         12.0|
|2014|   09|         15.0|
|2016|   04|          7.0|
|2015|   05|          9.0|
|2015|   07|          7.0|
|2014|   01|          7.0|
|2016|   07|          9.0|
|2016|   06|         12.0|
|2014|   02|          3.0|
|2015|   08|         19.0|
|2014|   08|          9.0|
|2014|   10|         14.0|
|2015|   02|          8.0|
|2016|   02|          6.0|
|2016|   03|          7.0|
+----+-----+-------------+
only showing top 20 rows



In [ ]:
#resample같은 기능은 없음